## Import

In [1]:
from config import car_parameter_fname
from config import reference_parameter_fname

import sys
sys.path.append('../')
from subword_df_handler import SubwordDocumentFrequencyHandler
from subword_df_handler import get_positive_words
from subword_df_handler import pprint_word_list

## Step 1. Extract positive subwords

In [2]:
car_subword_handler = SubwordDocumentFrequencyHandler(car_parameter_fname)
car_statistics = car_subword_handler.get_df_distribution_statistics_of_all_words()

ref_subword_handler = SubwordDocumentFrequencyHandler(reference_parameter_fname)
ref_statistics = ref_subword_handler.get_df_distribution_statistics_of_all_words()
car_positive, ref_positive, car_filtered = get_positive_words(car_statistics, ref_statistics)
print(len(car_filtered))

 computing (std/mean, mean, max/mean, argmax) was done.        
 computing (std/mean, mean, max/mean, argmax) was done.        
3614


In [3]:
for word in '엔진 엔진오일 차량 차대 중고 시동 배터리 유리'.split():
    print(word, word in car_statistics, word in car_filtered)

엔진 True True
엔진오일 True True
차량 True True
차대 True True
중고 True True
시동 True True
배터리 True True
유리 True False


In [4]:
pprint_word_list(list(car_filtered.keys()))

     튜닝카	      ES	   이노베이션	     12인	     세라토
     508	    뉴쎄라토	    머플러팁	   000Km	     삼성이
     183	      뉴E	    36개월	   알아봅시다	     740
      케미	    루프박스	   열차단썬팅	     사제휠	     잡소리
     파워트	    중고차의	      라프	    완료하여	     원스톱
      흠집	     미니밴	    싶습니다	   아우디A4	      후미
      대덕	      자차	     030	      앰프	   받아보실수
     흠잡을	    조작하기	      신품	      cr	     시승차
     156	   EF소나타	   헤드레스트	    차량이지	      시내
     모델이	      GR	     138	   중고타이어	      ne
      옥션	    대쉬보드	    작업문의	     드라마	    좋은차량
     플래그	     169	     아이신	     상태에	    등록되어
    운전자분	      전고	   일산화탄소	     보증해	    망설이지
     실내도	    통풍시트	   BMW52	     국내외	      검수
    서스펜션	    잔유제거	     다본다	      신대	     gdi
     세단에	      하부	     변속이	     고급감	      장성
   거래되고있	      용량	     알로이	     수리비	    s500
      on	      S8	    view	    520d	  뉴EF소나타
      X3	    차대번호	      액티	     리베로	    고압펌프
     베스타	     345	     영업용	     911	   스테이츠맨
     지원하	     테라칸	 로체이노베이션	     클럽맨	    차량들이
      복사	   수입타이어	     pcd	     공간을	      DM
    뉴마티즈	 

## Step 2. Extract category - sensitive subwords

In [37]:
def get_category_sensitive_words(pos_handler,
                                 pos_statistics,
                                 pos_min_df_nstd=2.5,
                                 pos_df_max_mean=0.01,
                                 pos_min_of_df_mean_ratio=3
                                ):
    word_by_category = [[] for _ in range(pos_handler.num_categories)]
    for subword, (nstd, mean, topmean, max_sensitive_category) in pos_statistics.items():
        if nstd < pos_min_df_nstd:
            continue
        if mean > pos_df_max_mean:
            continue
        idx = pos_handler.encode(subword)
        df_dist = pos_handler.subword_slot[idx,:]
        sensitive_categories = [i for i, r in enumerate(df_dist/mean) if r >= pos_min_of_df_mean_ratio]
        for c in sensitive_categories:
            word_by_category[c].append(subword)
    return word_by_category

sensitive_words_by_category = get_category_sensitive_words(car_subword_handler,
                                                           car_statistics,
                                                           pos_min_df_nstd=3,
                                                           pos_min_of_df_mean_ratio=4
                                                          )
len(sensitive_words_by_category)

27

In [39]:
from utils import read_list
from utils import remove_alphabet_number_comb
index2category = read_list('./car_index.txt')

for c, words in enumerate(sensitive_words_by_category):
    print('{}\ncategory ({}), {}'.format('-'*50,c, index2category[c]))
    pprint_word_list(remove_alphabet_number_comb(words))

--------------------------------------------------
category (0), A6
   아이패드3	    A6블록	    드니프로	   부육셰히르	 Sportiv
      Vf	    에스파뇰	 제조아우디차명	 bigfile	   60GHz
   66GHz	   아이패드2	 아우디A6차량	     Plo	     출시국
    로코모티	     크레테	    A6차량	    가람마을	    미사강변
  아우디뉴A6	      하그	    RegN	    입주예정	     로스토
   아이팟터치	   00GHz	   50GHz	     A6칩	  Olimpi
   30GHz	   10GHz	    쿼드코어	   뉴아이패드	 Tallinn
   70GHz	   광명역세권	     운정역	     코어i	    byte
   아이폰5C	      알메	    A6모델	     홈버튼	   20GHz
     아라우	 아우디A6중고	   우니베르시	   아이폰5에	   아이폰4S
     A6에	     A6블	     레티나	   80GHz	   40GHz
--------------------------------------------------
category (1), BMW5
 비엠더블류네비	  원동기형식N	    komm	  우수중소기업	   차명BMW
 BMW바바리안	 차대번호WBA	    welt	     마리엔	 뉴5시리즈모델
    휠코리아	 제조BMW차명	 차명BMW52	    DTM에	     코아스
   투자컨설팅	 원동기형식N5	     d가격	     비머몰	   F10가격
   님펜부르크	    와인에셋	 장외주식114
--------------------------------------------------
category (2), BMW
    komm	 BMW바바리안	    welt	     마리엔	    휠코리아
 제조BMW차명	    DTM에	     비머몰	   문학사상사	   님펜부르크
-

## Step 3. Extract category - specific subwords

## To be reviewed

In [ ]:
def is_co_occurred(wc, wf, doc):
    return 1 if ' '+wc in ' '+doc and ' '+wf in ' '+doc else 0

In [ ]:
def calculate_cooccurrence_document_frequency_ratio(category_sensitive_words_list, positive_words, subword2index, corpus_directory):
    """
    Arguments:
    ----------
        category_sensitive_words_list: list of set of str
            list[category_index]: [{word, word, ....}, {word, word, ....}, ...]
            len(category_sensitive_words_list) is equal number of categories
        positive_words: set of str
        
    Returns:
    ----------
        list of sparse matrices
    """
    
    list_of_sparse_matrices = []    
    
    from collections import defaultdict
    
    for c, category_sensitive_words in enumerate(category_sensitive_words_list):
        if len(category_sensitive_words) == 0:
            print('category = {}, There are no category sensitive words here'.format(c))
            continue
        wc_occurrence = defaultdict(int) 
        cooccurrence = defaultdict(lambda: defaultdict(int))
        corpus_fname = '{}/{}.txt'.format(corpus_directory, c)

        with open(corpus_fname, 'r', encoding='utf-8') as f:
            postings = f.readlines()
            
        process_time = time.time()
        for i_wc, wc in enumerate(category_sensitive_words):
            filtered_postings = [post for post in postings if wc in post]
            for wf in positive_words:     
                cooccur = sum([is_co_occurred(wc, wf, post) for post in filtered_postings]) / len(filtered_postings)
                cooccurrence[wc][wf] = cooccur
            if i_wc == 0:
                continue
            print('\r scanned {}/{} category sensitive words'.format(i_wc+1, len(category_sensitive_words)), flush=True, end='')
        
        process_time = time.time() - process_time
        print('\rcategory = {}, processing time = {}'.format(c, '%.2f sec' % process_time))
        
        
        from scipy.sparse import csr_matrix
        row_ind = []
        col_ind = []
        data = []
        for sensitive_word, positive_word_counter in cooccurrence.items():
            i = subword2index.get(sensitive_word, -1)
            if i == -1:
                continue
            for positive_word, cooccur in positive_word_counter.items():
                j = subword2index.get(positive_word, -1)
                if j == -1:
                    continue
                row_ind.append(i)
                col_ind.append(j)
                data.append(cooccur)
            
        csr_matrix((data, (row_ind, col_ind)))
    
    list_of_sparse_matrices.append(csr_matrix)
        
    return list_of_sparse_matrices

In [ ]:
list_of_sparse_cooccurrence_matrices = calculate_cooccurrence_document_frequency_ratio(category_sensitive_words_list = category_sensitive_words_list,
                                                                                       positive_words = filtered_car_words,
                                                                                       subword2index = positive_corpus.subword2index,
                                                                                       corpus_directory = '../corpus_norm/')